# GP Cartpole Service

This notebook uses the functions from the `gprest` module to enable genetic programming for REST service behaviors. It interacts with REST server stubs created by the OpenAPI generator.

The previous notebook [`gp_cartpole_server.ipynb`](https://github.com/cdeck3r/DRL4REST/notebooks/gp_cartpole_server.ipynb) shows the GP approach applied to the [cartpole](https://en.wikipedia.org/wiki/Inverted_pendulum) example for the `GET /api/v1/cart` request. The functions from this notebook are now a part of the `gprest` module.

This notebook implements the following functionalities:

1. It reproduces the results from `gp_cartpole_server.ipynb` using `gprest` module.
1. It uses the `gprest` module for implementing the similar `GET /api/v1/pole` request.
1. It introduces and tests the `PUT /api/v1/cart` request where a clients transmits a Direction object to the server to receive an updated the `Cart` object as response.

As a result, this notebook completes the implementation of the cartpole OpenAPI interface.


## Install required Modules

In [1]:
%%bash
# define project environment
PROJECT_DIR="/DRL4REST"
OPENAPI_SERVER_DIR="$PROJECT_DIR/openapi/cartpole/python-flask"

# install requirements
cd "$OPENAPI_SERVER_DIR" || exit
pip install -r requirements.txt || exit
pip install -r test-requirements.txt || exit

# install other requirements
pip install Werkzeug==0.16.1 || exit

Ignoring connexion: markers 'python_version == "3.5"' don't match your environment
Ignoring connexion: markers 'python_version == "3.4"' don't match your environment
Ignoring connexion: markers 'python_version <= "2.7"' don't match your environment


In [2]:
import random
import numpy
from functools import partial

In [3]:
# DEAP: python genetic algorithm library
from deap import algorithms
from deap import base
from deap import creator
from deap import tools
from deap import gp

In [4]:
# set path to import code generated by OpenAPI
import sys
sys.path.append("/DRL4REST/openapi/cartpole/python-flask")
sys.path.append("/DRL4REST/src")

In [5]:
from cartpole.gprest.server_model import CartpoleServer
from cartpole.gprest.monkey_patching import MonkeyPatching
from cartpole.gprest.gp_test_default_controller import GP_TestDefaultController

## Genetic Program Implementation

The GP program bases upon the Artificial Ant Problem from the [DEAP example site](https://deap.readthedocs.io/en/master/examples/gp_ant.html). The implementation consists of

* generic functions for the control flow
* controller functions interacting with the ServerModel
* `evaluate()` function to report the controller's fitness

### Generic Functions

These are generic functions, e.g. for control flow and concatenating functions with each other.

In [6]:
def progn(*args):
    for arg in args:
        arg()

def prog2(out1, out2): 
    return partial(progn,out1,out2)

def prog3(out1, out2, out3):     
    return partial(progn,out1,out2,out3)

def _if_then_else(condition, out1, out2):
    if condition() is not None:
        out1()
    else:
        out2()

def if_then_else(condition, out1, out2):
    return partial(_if_then_else, condition, out1, out2)

def _if_then(condition, out1):
    if condition() is not None:
        out1()
        
def if_then(condition, out1):
    return partial(_if_then, condition, out1)

### Controller Functions

These are the functions the GP algorithm strings together utilizing the generic functions from above. The created program forms an individual and is tested by the `evaluate()` function. The controller functions utilize the CartpoleServer's CRUD functions.

In [7]:
cps = CartpoleServer
cps.reset()

In [8]:
from functools import partial

# cart_get() controller must return something when done
# this function becomes the new root for the GP tree created from the pset above
# it runs the GP tree first and returns the Cart afterwards
def return_cart_get(gp_tree):
    def run_first_return_second(f1,f2):
        f1()
        return f2()
    
    return partial(run_first_return_second, gp_tree, cps.read_cart)

In [9]:
from functools import partial

# cart_pole_get() controller must return something when done
# this function becomes the new root for the GP tree created from the pset above
# it runs the GP tree first and returns the Pole afterwards
def return_cart_pole_get(gp_tree):
    def run_first_return_second(f1,f2):
        f1()
        return f2()
    
    return partial(run_first_return_second, gp_tree, cps.read_pole)

In [10]:
from functools import partial
from flask import Request
from flask import globals

# cart_put() controller must return something when done
# this function becomes the new root for the GP tree created from the pset above
#
# It runs the following command sequence:
# 1. store payload data in CartpoleServer
# 2. run GP tree 
# 3. return Cart from CartpoleServer
def return_cart_put(gp_tree):

    def store_json():
        req = globals._lookup_req_object('request')
        if req.is_json:
            cps._from_client = req.get_json()
        
    def run_first_return_second(f1,f2):
        store_json()
        f1()
        return f2()
    
    return partial(run_first_return_second, gp_tree, cps.read_cart)

### Compile Primitive Set

Compile the primitive set `pset` consisting of the generic funtions and ServerModel's controller functions.

In [11]:
def create_generic_pset():
    pset = gp.PrimitiveSet("MAIN", 0)
    pset.addPrimitive(if_then_else, 3)
    pset.addPrimitive(if_then, 2)
    pset.addPrimitive(prog2, 2)
    pset.addPrimitive(prog3, 3)
    return pset

# initial add generic functions
pset = create_generic_pset()

# let's add the ServerModel's CRUD functions
import cartpole.gprest.server_model as sm
pset = sm.add_CRUD_pset(pset, cps, 'cart')

### `evaluate()` Function

It patches the endpoint with the new controller function and runs the controller unittest. Evaluation reports the controller's fitness to guide the next program evolution.

In [12]:
from cartpole.gprest.monkey_patching import gpc

#### `GET /v1/api/cart` Request

In [13]:
# initial add generic functions
pset = create_generic_pset()

# let's add the ServerModel's CRUD functions
import cartpole.gprest.server_model as sm
pset = sm.add_CRUD_pset(pset, cps, 'cart')

In [14]:
import types
import json

# evaluate()
def evaluate_cart_get(individual):
   
    def add_func2pset(expr, pset, func, num_args):
        pset_cloned = toolbox.clone(pset)
        pset_cloned.addPrimitive(func, num_args)
        prim_func = gp.Primitive(func.__name__, [object], object)
        expr_func = [prim_func] + expr
        return expr_func, pset_cloned

    # we add the return_cart_get() als new root of the individual
    # togehter they form the gp_controller

    assert isinstance(return_cart_get, types.FunctionType), 'return_cart_get() not defined'
    
    # 1. dismantle the individual to get the expressions
    expr = list(individual)

    # 2. add return_cart_get als new root
    expr_return, pset_return = add_func2pset(expr=expr,
                                             pset=pset,
                                             func=return_cart_get, 
                                             num_args=1)
    # 3. rebuild individual (type: PrimitiveTree) from expressions 
    individual_return = gp.PrimitiveTree(expr_return)
    
    # 4. .. and compile tree to functional Python code 
    gp_controller = gp.compile(individual_return, pset_return)

    # store the controller
    gpc.set_controller_func(gp_controller)
    #gpc.set_controller_func(testcase_all_correct)

    # Replace the default controller with the gp_controller 
    url_path = '/api/v1/cart'
    gp_test = GP_TestDefaultController()
    ret = gp_test.endpoint_config(url_path, 'get', gpc.gp_controller_func)
    # ... and test
    cps.reset()
    gp_test.reset_score()
    
    #gp_test.test_cart_get()
    # run the test several times and expect the same as the first one
    for i in range(5):
        gp_test.safe_test_cart_get()
    
    # retrieve score 
    fitness = gp_test.score
        
    return fitness,

#### `GET /v1/api/pole` Request

In [15]:
# initial add generic functions
pset = create_generic_pset()

# let's add the ServerModel's CRUD functions
import cartpole.gprest.server_model as sm
pset = sm.add_CRUD_pset(pset, cps, 'pole')

In [16]:
import types
import json

# evaluate()
def evaluate_cart_pole_get(individual):
   
    def add_func2pset(expr, pset, func, num_args):
        pset_cloned = toolbox.clone(pset)
        pset_cloned.addPrimitive(func, num_args)
        prim_func = gp.Primitive(func.__name__, [object], object)
        expr_func = [prim_func] + expr
        return expr_func, pset_cloned

    # we add the return_cart_get() als new root of the individual
    # togehter they form the gp_controller

    assert isinstance(return_cart_pole_get, types.FunctionType), 'return_cart_pole_get() not defined'
    
    # 1. dismantle the individual to get the expressions
    expr = list(individual)

    # 2. add return_cart_get als new root
    expr_return, pset_return = add_func2pset(expr=expr,
                                             pset=pset,
                                             func=return_cart_pole_get, 
                                             num_args=1)
    # 3. rebuild individual (type: PrimitiveTree) from expressions 
    individual_return = gp.PrimitiveTree(expr_return)
    
    # 4. .. and compile tree to functional Python code 
    gp_controller = gp.compile(individual_return, pset_return)

    # store the controller
    gpc.set_controller_func(gp_controller)
    #gpc.set_controller_func(testcase_all_correct)

    # Replace the default controller with the gp_controller 
    url_path = '/api/v1/cart/pole'
    gp_test = GP_TestDefaultController()
    ret = gp_test.endpoint_config(url_path, 'get', gpc.gp_controller_func)
    # ... and test
    cps.reset()
    gp_test.reset_score()
    
    #gp_test.test_cart_get()
    # run the test several times and expect the same as the first one
    for i in range(5):
        gp_test.safe_test_cart_pole_get()
    
    # retrieve score 
    fitness = gp_test.score

    return fitness,

#### `PUT /v1/api/cart` Request

In this request, the client serializes a `Direction` object as json and adds it as payload data in a PUT request. Due to the closure property of the functions used for genetic programming, the receiving `return_cart_put()` controller, when called by flask, needs handle the payload data separatly.

The `return_cart_put()` controller stores the json encoded payload data within the `CartpoleServer`. The `update_cart()` method from the `CartpoleServer` processes the payload. It de-serializes the json data as `Direction` object, which resembles the original one transmitted by the PUT request, and updates the `Cart` object. Finally, it deletes the json data from the `CartpoleServer`. As per specification, the client receives the recent `Cart` object as a response. The sequence diagram below illustrates these steps.

![PUT request sequence diagram](http://www.plantuml.com/plantuml/png/1S4z4e9030RGg-W5DeLMrjfPE8Wr0-4ROTXD93_ZzNdll0hNd95MJql4psz4DlxBTuqbftGOgmGa2ciZdi7RnUZUFQ_TuzrJ6prW91gOL3UdrwSV37l2Ot3i9Liz0000)

As a consequence, `update_cart()` sucessfully processes the payload data can only once. A second call to `update_cart()` will let the `Cart` object unchanged, because of the deleted payload data in the previous successful run. If `update_cart()` encounters an error, it will stop and return to the caller. The payload will remain as it is for an other function to process. 

In [17]:
# initial add generic functions
pset = create_generic_pset()

# let's add the ServerModel's CRUD functions
import cartpole.gprest.server_model as sm
pset = sm.add_CRUD_pset(pset, cps, 'cart')

In [18]:
import types
import json
import random
from openapi_server.models import Direction

# evaluate()
def evaluate_cart_put(individual):
   
    def add_func2pset(expr, pset, func, num_args):
        pset_cloned = toolbox.clone(pset)
        pset_cloned.addPrimitive(func, num_args)
        prim_func = gp.Primitive(func.__name__, [object], object)
        expr_func = [prim_func] + expr
        return expr_func, pset_cloned

    # we add the return_cart_get() als new root of the individual
    # togehter they form the gp_controller

    assert isinstance(return_cart_put, types.FunctionType), 'return_cart_put() not defined'
    
    # 1. dismantle the individual to get the expressions
    expr = list(individual)

    # 2. add return_cart_get als new root
    expr_return, pset_return = add_func2pset(expr=expr,
                                             pset=pset,
                                             func=return_cart_put, 
                                             num_args=1)
    # 3. rebuild individual (type: PrimitiveTree) from expressions 
    individual_return = gp.PrimitiveTree(expr_return)
    
    # 4. .. and compile tree to functional Python code 
    gp_controller = gp.compile(individual_return, pset_return)

    # store the controller
    gpc.set_controller_func(gp_controller)

    # Replace the default controller with the gp_controller 
    url_path = '/api/v1/cart'
    gp_test = GP_TestDefaultController()
    ret = gp_test.endpoint_config(url_path, 'put', gpc.gp_controller_func)
    # ... and test
    cps.reset()
    gp_test.reset_score()
    
    # run PUT request several times each with random directions 
    for i in range(5):
        direction = Direction(direction=random.choice(['left', 'right']))
        gp_test.safe_test_cart_put(direction)
    
    # retrieve score 
    fitness = gp_test.score
    
    return fitness,

## Assemble and Configure the GP

We configure the GP to solve a minimization problem, `FitnessMin`. The `evaluate()` function returns the cumulative sum of the responses' status codes when testing the controller as fitness. 

In [19]:
# configure the parameters
creator.create("FitnessMin", base.Fitness, weights=(-1.0,))
creator.create("Individual", gp.PrimitiveTree, fitness=creator.FitnessMin)

# Attribute generator
toolbox = base.Toolbox()
toolbox.register("expr_init", gp.genFull, pset=pset, min_=1, max_=2)

# Structure initializers
toolbox.register("individual", tools.initIterate, creator.Individual, toolbox.expr_init)
toolbox.register("population", tools.initRepeat, list, toolbox.individual)

Register the GP operators. They mate, mutate and change in various ways the individuals made of programs. Further individuals are the result from the GP algorithm using these GP operators.

In [20]:
toolbox.register("evaluate", evaluate_cart_put)
toolbox.register("select", tools.selTournament, tournsize=7)
toolbox.register("mate", gp.cxOnePoint)
toolbox.register("expr_mut", gp.genFull, min_=0, max_=2)
toolbox.register("mutate", gp.mutUniform, expr=toolbox.expr_mut, pset=pset)

## Run the GP Algorithm

In [39]:
random.seed(456)

pop = toolbox.population(n=300)
hof = tools.HallOfFame(1)
stats = tools.Statistics(lambda ind: ind.fitness.values)
stats.register("avg", numpy.mean)
stats.register("std", numpy.std)
stats.register("min", numpy.min)
stats.register("max", numpy.max)

pop, log = algorithms.eaSimple(pop, toolbox, 0.5, 0.3, 4, stats, halloffame=hof)

gen	nevals	avg    	std    	min	max 
0  	300   	3232.33	2183.81	199	4999
1  	197   	1327   	1827.97	199	4999
2  	201   	1160.33	1811.69	199	4999
3  	171   	921.667	1629.69	199	4999
4  	200   	1204.33	1851.69	199	4999


## Display and Plot

Plot the individual. It is a tree structure contained in the halloffame `hof` object.

`hof[0]` indicates the individual tree structure.

In [40]:
str(hof[0])

'prog3(if_then(create_cart, create_cart), if_then_else(create_cart, update_cart, update_cart), if_then(read_cart, update_cart))'

## Unit Testing Example Controllers

### cart_pole_get() Controller Examples

In [21]:
# initial add generic functions
pset = create_generic_pset()

# let's add the ServerModel's CRUD functions
import cartpole.gprest.server_model as sm
pset = sm.add_CRUD_pset(pset, cps, 'pole')

In [22]:
import unittest
import types
from openapi_server.models import Pole

class Test_ExamplePoleController(unittest.TestCase):

    @classmethod
    def setUpClass(cls):
        super().setUpClass()
        assert isinstance(toolbox, base.Toolbox), 'toolbox not defined'
        cls.assertIsNotNone(toolbox.individual(), 'toolbox.individual() does not work')
        assert cps == CartpoleServer, 'cps is not a CartpoleServer'
    
    def setUp(self):
        self.assertIsInstance(pset, gp.PrimitiveSet, 'pset not defined')
        self.assertIsInstance(toolbox, base.Toolbox, 'toolbox not defined')
        # Setup cps 
        cps.reset()
        self.assertIsNone(cps.read_pole())
        
    def test_delete_create(self):
        gp_ctrl = 'prog2(delete_pole, create_pole)'
        ind = gp.PrimitiveTree.from_string(gp_ctrl, pset)
        f = gp.compile(ind, pset)
        f()
        self.assertIsInstance(cps.read_pole(), Pole)

    def test_create_delete(self):
        gp_ctrl = 'prog2(create_pole, delete_pole)'
        ind = gp.PrimitiveTree.from_string(gp_ctrl, pset)
        f = gp.compile(ind, pset)
        f()
        self.assertNotIsInstance( cps.read_pole(), Pole )
        self.assertIsNone( cps.read_pole() )

    def test_double_create(self):
        gp_ctrl = 'create_pole'
        ind = gp.PrimitiveTree.from_string(gp_ctrl, pset)
        f = gp.compile(ind, pset)
        # 1st run
        f()
        self.assertIsInstance(cps.read_pole(), Pole)
        pole1 = cps.read_pole()
        # 2nd run
        f()
        self.assertIsInstance(cps.read_pole(), Pole)
        pole2 = cps.read_pole()
        # Poles are different
        self.assertNotEqual(pole1, pole2)
        
    def test_read(self):
        gp_ctrl = 'read_pole'
        ind = gp.PrimitiveTree.from_string(gp_ctrl, pset)
        f = gp.compile(ind, pset)
        f()
        self.assertIsNone( cps.read_pole() )
        
    def test_create_double_read(self):
        gp_ctrl = 'create_pole'
        ind = gp.PrimitiveTree.from_string(gp_ctrl, pset)
        f = gp.compile(ind, pset)
        f()
        pole1 = cps.read_pole()
        self.assertIsInstance( pole1, Pole )
        
        gp_ctrl = 'prog2(read_pole, read_pole)'
        ind = gp.PrimitiveTree.from_string(gp_ctrl, pset)
        f = gp.compile(ind, pset)
        f()
        pole2 = cps.read_pole()
        self.assertIsInstance( pole2, Pole )
        self.assertEqual(pole1, pole2)
        
    def test_cart_pole_get_controller(self):
        gp_ctrl = 'prog2(if_then_else(read_pole, update_pole, create_pole), if_then(update_pole, create_pole))'
        ind = gp.PrimitiveTree.from_string(gp_ctrl, pset)
        f = gp.compile(ind, pset)
        f()
        pole1 = cps.read_pole()
        self.assertIsInstance( pole1, Pole )
        f()
        pole2 = cps.read_pole()
        self.assertIsInstance( pole2, Pole )
        self.assertEqual(pole1, pole2)


In [23]:
suite = unittest.defaultTestLoader.loadTestsFromTestCase(Test_ExamplePoleController)
unittest.TextTestRunner().run(suite)

......
----------------------------------------------------------------------
Ran 6 tests in 0.006s

OK


<unittest.runner.TextTestResult run=6 errors=0 failures=0>

## Unit Testing `evaluate()`


In [24]:
# initial add generic functions
pset = create_generic_pset()

# let's add the ServerModel's CRUD functions
import cartpole.gprest.server_model as sm
pset = sm.add_CRUD_pset(pset, cps, 'pole')

In [25]:
import unittest
import types


# return_cart_pole_get() controller must return something
def return_cart_pole_get(gp_tree):
    def run_first_return_second(f1,f2):
        f1()
        return f2()
    
    return partial(run_first_return_second, gp_tree, cps.read_pole)


class Test_EvaluateGetPole(unittest.TestCase):
        
    @classmethod
    def setUpClass(cls):
        super().setUpClass()
        assert cps == CartpoleServer, 'cps is not a CartpoleServer'
    
    def setUp(self):
        self.assertIsInstance(pset, gp.PrimitiveSet, 'pset not defined')
        self.assertIsInstance(return_cart_pole_get, types.FunctionType, 'return_cart_pole_get() not defined')
        
        self.ok_score = 200
        self.assert_score = 1000
        self.num_tests = 5
        
    def run_eval(self, gp_ctrl):
        ind = gp.PrimitiveTree.from_string(gp_ctrl, pset)
        fitness = evaluate_cart_pole_get(ind)[0]
        return fitness
        
    def test_create_controller(self):
        test_gp_controller = 'create_pole'
        fitness = self.run_eval(test_gp_controller)
        self.assertEqual(fitness, self.num_tests*self.ok_score-1)
    
    def test_read_controller(self):
        test_gp_controller = 'read_pole'
        fitness = self.run_eval(test_gp_controller)
        self.assertEqual(fitness, self.num_tests*self.assert_score-1)

    def test_create_read_controller(self):
        test_gp_controller = 'prog2(create_pole, read_pole)'
        fitness = self.run_eval(test_gp_controller)
        self.assertEqual(fitness, self.num_tests*self.ok_score-1)

    def test_read_create_controller(self):
        test_gp_controller = 'prog2(read_pole, create_pole)'
        fitness = self.run_eval(test_gp_controller)
        self.assertEqual(fitness, self.num_tests*self.ok_score-1)

    def test_create_delete_controller(self):
        test_gp_controller = 'prog2(create_pole, delete_pole)'
        fitness = self.run_eval(test_gp_controller)
        self.assertEqual(fitness, self.num_tests*self.assert_score-1)
        
    def test_delete_create_controller(self):
        test_gp_controller = 'prog2(delete_pole, create_pole)'
        fitness = self.run_eval(test_gp_controller)
        self.assertEqual(fitness, self.num_tests*self.ok_score-1)

    # this controller was falsely scored by with a 199 fitness, which implies
    # the cart_get creates and repeatedly returns the created Cart object 
    @unittest.expectedFailure
    def test_false_cart_pole_get_controller(self):
        test_gp_controller = 'if_then_else(create_pole, create_pole, read_pole)'
        fitness = self.run_eval(test_gp_controller)
        self.assertEqual(fitness, self.ok_score-1)

    # this is the true controller
    def test_cart_pole_get_controller(self):
        test_gp_controller = 'prog2(if_then_else(read_pole, update_pole, create_pole), if_then(update_pole, create_pole))'
        fitness = self.run_eval(test_gp_controller)
        self.assertEqual(fitness, self.ok_score-1)
        

In [26]:
suite = unittest.defaultTestLoader.loadTestsFromTestCase(Test_EvaluateGetPole)
unittest.TextTestRunner().run(suite)

.....x..
----------------------------------------------------------------------
Ran 8 tests in 0.439s

OK (expected failures=1)


<unittest.runner.TextTestResult run=8 errors=0 failures=0>

## Unit Testing `PUT /v1/api/cart` Request

In [27]:
cps = CartpoleServer
cps.reset()

In [28]:
# initial add generic functions
pset = create_generic_pset()

# let's add the ServerModel's CRUD functions
import cartpole.gprest.server_model as sm
pset = sm.add_CRUD_pset(pset, cps, 'cart')

In [29]:
from functools import partial
from flask import Request
from flask import globals

# cart_put() controller must return something when done
# this function becomes the new root for the GP tree created from the pset above
#
# It runs the following command sequence:
# 1. store payload data in CartpoleServer
# 2. run GP tree 
# 3. return Cart from CartpoleServer
def return_cart_put(gp_tree):

    def store_json():
        req = globals._lookup_req_object('request')
        if req.is_json:
            cps._from_client = req.get_json()
        
    def run_first_return_second(f1,f2):
        store_json()
        f1()
        return f2()
    
    return partial(run_first_return_second, gp_tree, cps.read_cart)

In [42]:
import unittest
import types
import random
from openapi_server.models import Direction
from openapi_server.models import Cart

class Test_PUTRequestController(unittest.TestCase):

    @classmethod
    def setUpClass(cls):
        super().setUpClass()
        assert isinstance(toolbox, base.Toolbox), 'toolbox not defined'
        cls.assertIsNotNone(toolbox.individual(), 'toolbox.individual() does not work')
        assert cps == CartpoleServer, 'cps is not a CartpoleServer'
    
    def setUp(self):
        self.assertIsInstance(pset, gp.PrimitiveSet, 'pset not defined')
        self.assertIsInstance(toolbox, base.Toolbox, 'toolbox not defined')
        # Setup cps 
        cps.reset()
        self.assertIsNone(cps.read_cart())

    # replaces the original view_func with the test_gp_controller and
    # performs the PUT request
    def _cart_put_request(self, test_gp_controller, direction):

        def add_func2pset(expr, pset, func, num_args):
            pset_cloned = toolbox.clone(pset)
            pset_cloned.addPrimitive(func, num_args)
            prim_func = gp.Primitive(func.__name__, [object], object)
            expr_func = [prim_func] + expr
            return expr_func, pset_cloned

        # we add the return_cart_get() als new root of the individual
        # togehter they form the gp_controller
        assert isinstance(return_cart_put, types.FunctionType), 'return_cart_put() not defined'
        # 1. dismantle the individual to get the expressions
        expr = list(test_gp_controller)
        # 2. add return_cart_get als new root
        expr_return, pset_return = add_func2pset(expr=expr,
                                                 pset=pset,
                                                 func=return_cart_put, 
                                                 num_args=1)
        # 3. rebuild individual (type: PrimitiveTree) from expressions 
        individual_return = gp.PrimitiveTree(expr_return)
        # 4. .. and compile tree to functional Python code 
        gp_controller = gp.compile(individual_return, pset_return)
        # store the controller
        gpc.set_controller_func(gp_controller)

        # Replace the default controller with the gp_controller 
        url_path = '/api/v1/cart'
        gp_test = GP_TestDefaultController()
        ret = gp_test.endpoint_config(url_path, 'put', gpc.gp_controller_func)
        # ... and test
        cps.reset()
        gp_test.reset_score()
        resp = gp_test.test_cart_put(direction)

        return resp
    
    def test_direction_right(self):
        direction = Direction('right')
        test_gp_controller_str = 'prog2(create_cart, update_cart)'
        test_gp_controller = gp.PrimitiveTree.from_string(test_gp_controller_str, pset)
        
        # hammer request
        for i in range(40):
            response = self._cart_put_request(test_gp_controller, direction)
            cart = Cart.from_dict(response.json)
            self.assertIsInstance( cart, Cart )
            self.assertEqual('right', cart.direction)

    def test_direction_left(self):
        direction = Direction('left')
        test_gp_controller_str = 'prog2(create_cart, update_cart)'
        test_gp_controller = gp.PrimitiveTree.from_string(test_gp_controller_str, pset)
        
        # hammer request
        for i in range(40):
            response = self._cart_put_request(test_gp_controller, direction)
            cart = Cart.from_dict(response.json)
            self.assertIsInstance( cart, Cart )
            self.assertEqual('left', cart.direction)

    def test_direction_random(self):
        # hammer request
        for i in range(40):
            random_direction = random.choice(['left', 'right'])
            direction = Direction(random_direction)
            test_gp_controller_str = 'prog2(create_cart, update_cart)'
            test_gp_controller = gp.PrimitiveTree.from_string(test_gp_controller_str, pset)
            
            # issue request
            response = self._cart_put_request(test_gp_controller, direction)
            cart = Cart.from_dict(response.json)
            self.assertIsInstance( cart, Cart )
            self.assertEqual(random_direction, cart.direction)
    
    # yields bad request
    def test_direction_empty(self):
        direction = Direction('')
        test_gp_controller_str = 'prog2(create_cart, update_cart)'
        test_gp_controller = gp.PrimitiveTree.from_string(test_gp_controller_str, pset)
        
        # issue request
        try:
            response = self._cart_put_request(test_gp_controller, direction)
        except AssertionError as ae:
            ae_str = str(ae)
            self.assertTrue(ae_str.startswith('400 != 200'))

    # yields bad request
    def test_direction_none(self):
        direction = Direction()
        test_gp_controller_str = 'prog2(create_cart, update_cart)'
        test_gp_controller = gp.PrimitiveTree.from_string(test_gp_controller_str, pset)
        
        # issue request
        try:
            response = self._cart_put_request(test_gp_controller, direction)
        except AssertionError as ae:
            ae_str = str(ae)
            self.assertTrue(ae_str.startswith('400 != 200'))

    # yields 204 no content
    def test_direction_update_only(self):
        direction = Direction('left')
        test_gp_controller_str = 'update_cart'
        test_gp_controller = gp.PrimitiveTree.from_string(test_gp_controller_str, pset)
        
        # issue request
        try:
            response = self._cart_put_request(test_gp_controller, direction)
        except AssertionError as ae:
            ae_str = str(ae)
            self.assertTrue(ae_str.startswith('204 != 200'))

    def test_direction_create_only(self):
        direction = Direction('left')
        test_gp_controller_str = 'create_cart'
        test_gp_controller = gp.PrimitiveTree.from_string(test_gp_controller_str, pset)

        # hammer request
        for i in range(40):        
            response = self._cart_put_request(test_gp_controller, direction)
            cart = Cart.from_dict(response.json)
            self.assertIsInstance( cart, Cart )
            try:
                self.assertEqual('right', cart.direction)
                break
            except AssertionError:
                pass
        # proves that loop did not run to the end
        self.assertTrue(i < 39)

    def test_direction_update_and_create(self):
        direction = Direction('left')
        test_gp_controller_str = 'prog2(update_cart, create_cart)'
        test_gp_controller = gp.PrimitiveTree.from_string(test_gp_controller_str, pset)

        # hammer request
        for i in range(40):        
            response = self._cart_put_request(test_gp_controller, direction)
            cart = Cart.from_dict(response.json)
            self.assertIsInstance( cart, Cart )
            try:
                #print('Counter: {0}, Direction: {1}'.format(i, cart.direction))
                self.assertEqual('right', cart.direction)
                break
            except AssertionError:
                pass
        # proves that loop did not run to the end
        self.assertTrue(i < 39)
        
    def test_candidates(self):
        def hammer_request(test_gp_controller_str):
            for i in range(200):
                random_direction = random.choice(['left', 'right'])
                direction = Direction(random_direction)
                test_gp_controller = gp.PrimitiveTree.from_string(test_gp_controller_str, pset)

                # issue request
                response = self._cart_put_request(test_gp_controller, direction)
                cart = Cart.from_dict(response.json)
                try:
                    self.assertIsInstance(cart, Cart)
                    self.assertEqual(random_direction, cart.direction)
                except:
                    self.fail('Candidate fails: {}'.format(test_gp_controller_str))
        
        # GP created controller
        test_gp_controller_str = 'prog3(prog3(delete_cart, update_cart, create_cart), if_then(create_cart, delete_cart), prog2(read_cart, update_cart))'
        hammer_request(test_gp_controller_str)
        
        test_gp_controller_str = 'prog3(create_cart, update_cart, update_cart)'
        hammer_request(test_gp_controller_str)
        
        test_gp_controller_str = 'prog3(if_then(create_cart, create_cart), if_then_else(create_cart, update_cart, update_cart), if_then(read_cart, update_cart))'
        hammer_request(test_gp_controller_str)


In [43]:
suite = unittest.defaultTestLoader.loadTestsFromTestCase(Test_PUTRequestController)
unittest.TextTestRunner().run(suite)

..http://localhost/api/v1/cart validation error: '' is not one of ['left', 'right'] - 'direction'
..http://localhost/api/v1/cart validation error: None is not one of ['left', 'right'] - 'direction'
.....
----------------------------------------------------------------------
Ran 9 tests in 26.030s

OK


<unittest.runner.TextTestResult run=9 errors=0 failures=0>